In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

%load_ext autoreload
%autoreload 2

from caltech256 import Caltech256
from deep_cnn_structures import VGG_fe, train_model, test_model, each_class_accuracy

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'visual': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
}
'''
# Replace Resize() with below in pytorch 0.2
transforms.Scale(256),
'''

data_dir = 'data/256_ObjectCategories'
#data_dir = '/datasets/Caltech256/256_ObjectCategories'

In [4]:
caltech256_train = Caltech256(data_dir, data_transforms['train'], train=True)
caltech256_test = Caltech256(data_dir, data_transforms['test'], train=False)

In [5]:
vgg_fe3 = VGG_fe(24, 28*28*256)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg_fe3.fc.parameters())

vgg_fe3 = nn.DataParallel(vgg_fe3)
vgg_fe3 = vgg_fe3.cuda()

In [6]:
vgg_fe3 = train_model(vgg_fe3, caltech256_train, criterion, optimizer, num_epochs=5, batch_size=16)

RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCStorage.cu:58

In [ ]:
test_model(vgg_fe3, caltech256_test)

In [ ]:
each_class_accuracy(vgg_fe3, caltech256_test)

In [5]:
vgg_fe4 = VGG_fe(34, 14*14*512)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg_fe4.fc.parameters())

vgg_fe4 = nn.DataParallel(vgg_fe4)
vgg_fe4 = vgg_fe4.cuda()

In [6]:
vgg_fe4 = train_model(vgg_fe4, caltech256_train, criterion, optimizer, num_epochs=5, batch_size=16)

0 epoch loss: 7.115856    accuracy: 20.715332%
1 epoch loss: 0.470705    accuracy: 92.236328%
2 epoch loss: 0.578577    accuracy: 92.431641%
3 epoch loss: 4.073687    accuracy: 73.730469%
4 epoch loss: 2.784355    accuracy: 85.107422%
Training comple in 13m, 29s


In [7]:
test_model(vgg_fe4, caltech256_test)

AttributeError: 'VGG_fe' object has no attribute 'features1'

In [ ]:
each_class_accuracy(vgg_fe4, caltech256_test)